In [18]:
import numpy 
numpy.random.seed(1337)

In [19]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

In [40]:
batch_size=128 
num_classes=37
epochs=60

In [21]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [22]:
os.chdir("E:");
path="E:/Dummy/Level_1";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [23]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(1080, 100, 100)
(1080,)


In [24]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [25]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(720, 100, 100, 1)
(720,)
(360, 100, 100, 1)
(360,)


In [26]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [34]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(32, (3, 3), padding='same')(inputs)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(7, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)

# x=Dropout(0.2)(x)#---------------------------------------------->0.3389(Epoch_24)
# x=Dense(num_classes)(x)
# x=SReLU()(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)

# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
s_re_lu_4 (SReLU)            (None, 100, 100, 32)      1280000   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 7)         2023      
_________________________________________________________________
s_re_lu_5 (SReLU)            (None, 50, 50, 7)         70000     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 7)         0         
__________

In [36]:
opt=keras.optimizers.Adam(lr=1e-3)

In [37]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [38]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [41]:
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               )

Train on 720 samples, validate on 360 samples
Epoch 1/60
720/720 [==============================] - 40s 55ms/step - loss: 3.4434 - acc: 0.1417 - val_loss: 3.2818 - val_acc: 0.1417

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 1-0.14.hdf5
Epoch 2/60
720/720 [==============================] - 38s 52ms/step - loss: 2.9946 - acc: 0.2806 - val_loss: 2.8090 - val_acc: 0.2167

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 2-0.22.hdf5
Epoch 3/60
720/720 [==============================] - 36s 50ms/step - loss: 2.3828 - acc: 0.3528 - val_loss: 2.2351 - val_acc: 0.2750

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 3-0.28.hdf5
Epoch 4/60
720/720 [==============================] - 35s 49ms/step - loss: 1.8520 - acc: 0.3833 - val_loss: 1.9944 - val_acc: 0.3000

Epoch 00004: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/

720/720 [==============================] - 35s 49ms/step - loss: 0.6961 - acc: 0.6694 - val_loss: 2.3360 - val_acc: 0.3139

Epoch 00034: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-34-0.31.hdf5
Epoch 35/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6550 - acc: 0.6875 - val_loss: 2.3590 - val_acc: 0.3083

Epoch 00035: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-35-0.31.hdf5
Epoch 36/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6743 - acc: 0.6889 - val_loss: 2.4457 - val_acc: 0.3139

Epoch 00036: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-36-0.31.hdf5
Epoch 37/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6626 - acc: 0.6722 - val_loss: 2.4356 - val_acc: 0.3000

Epoch 00037: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-37-0.30.hdf5
Epoch 38/60


In [42]:
score=model.evaluate(x_test, y_test)

360/360 [==============================] - 5s 14ms/step


In [43]:
score[1]

0.31111111111111112

In [44]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  2.96732283e-09   7.33897250e-05   1.34394912e-04 ...,   2.66780648e-02
    8.76068196e-04   1.60334966e-05]
 [  1.44327936e-08   7.52804699e-05   1.76889114e-06 ...,   3.31100804e-04
    2.47228331e-06   3.70038083e-06]
 [  1.68288066e-14   6.83017748e-12   4.22392486e-06 ...,   4.29017919e-14
    1.73891435e-12   9.95784700e-01]
 ..., 
 [  2.48181808e-10   4.18301882e-09   5.32046974e-01 ...,   2.20651927e-07
    5.14101713e-11   2.94603751e-12]
 [  1.45202583e-09   1.40393298e-04   9.44077598e-08 ...,   5.36014268e-04
    9.76784170e-01   1.83405264e-05]
 [  1.31457313e-07   1.75945665e-04   5.11757662e-07 ...,   8.26684106e-03
    1.18101276e-07   1.47840410e-06]]
[17 11 36 24 27 35  3  1 14 11 10 34 24 17  4 19 10 29  6 11 25  1 19 27 25
 25 26 11 27 23 14 11 13  6 18 34 18 21 30 14 25  8 12 27 36 21 20  3 31 25
 35  5 30 16 10 30 10  4 18 22 18 13 25 15 15 18  3 10 31 14  8 30  2 10 34
  1 25 23 31 34 22 13  9 32 18 18 19  5 29 35 26 18 23 25 20 28 13 29  3  4
 18 11 16  4  3 

In [45]:
model.load_weights("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 9-0.34.hdf5")

In [46]:
score=model.evaluate(x_test, y_test)

360/360 [==============================] - 4s 11ms/step


In [47]:
score[1]

0.34166666666666667

In [65]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  1.14446530e-09   4.88436781e-04   3.22940788e-04 ...,   2.62557454e-02
    2.00021155e-02   9.22519612e-05]
 [  2.33385777e-09   1.41028790e-02   4.43182907e-05 ...,   6.08796300e-03
    1.06983294e-03   6.54484174e-05]
 [  2.40366278e-18   1.82111660e-14   3.46898567e-04 ...,   5.09986289e-11
    7.53880902e-10   9.90875363e-01]
 ..., 
 [  2.79005031e-12   2.80058465e-09   1.52702257e-01 ...,   7.55173869e-06
    1.37824632e-08   1.12375531e-08]
 [  8.31410052e-10   8.04339349e-03   5.82458415e-06 ...,   1.78518402e-03
    7.58020878e-01   2.83352751e-03]
 [  5.43830971e-08   1.00230305e-02   1.14690674e-05 ...,   7.34316036e-02
    1.09331115e-04   5.95340280e-05]]
[17 11 36 25 27 35  3  1 33 11 10 12 10 14  5 18  8 28  6 13 25  1 19 27 25
 24 26 35 27 23 15 13 13  5 18 34 18 20 30 14 25 10 35 27 36 21 20  3 31 10
 34  5 30 14  9 30  8  5 19 22 20 13 24 14 14 18  2 10 31 14 24 30  5  8  1
  1 31 23 31 34 22 35  9 33 18 18 19  5 29 35 26 18 23 25 20 28 13 28  3  4
 20 14 17 26  3 

In [67]:
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[11  0  0 ...,  0  0  0]
 [ 0  1  6 ...,  0  0  0]
 [ 0  4  7 ...,  0  0  0]
 ..., 
 [ 0  0  0 ...,  4  0  0]
 [ 0  0  0 ...,  0  6  0]
 [ 0  0  0 ...,  0  0  5]]


In [66]:

print (classification_report(numpy.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00        11
          2       0.12      0.08      0.10        12
          3       0.50      0.54      0.52        13
          4       0.00      0.00      0.00        10
          5       0.10      0.40      0.16         5
          6       0.00      0.00      0.00        10
          7       0.33      0.08      0.12        13
          8       0.07      0.12      0.09         8
          9       0.09      0.08      0.09        12
         10       0.00      0.00      0.00         3
         11       0.18      0.18      0.18        11
         12       0.00      0.00      0.00        12
         13       0.10      0.14      0.12         7
         14       0.18      0.44      0.26         9
         15       0.00      0.00      0.00        12
         16       0.29      0.17      0.21        12
         17       0.58      0.54      0.56        13
         18       0.40      0.31      0.35   

In [69]:
import pandas

pandas.DataFrame(hist.history).to_csv("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/Figure/History.csv")        
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [70]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/Figure/Loss.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [71]:
pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/Figure/Accuracy.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()